# Path to dataset

In [2]:
import os
from google.colab import drive
drive.mount('/content/gdrive',force_remount=True)
path_dataset = '/content/gdrive/My Drive/Doutorado/Bases/GAMMA_training_data/multi-modality_images/'#path to dataset
os.chdir(path_dataset)

Mounted at /content/gdrive


# Setup

In [3]:
pip install -U tensorflow-addons

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 612.1/612.1 kB 6.1 MB/s eta 0:00:00


In [4]:
pip install optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 390.6/390.6 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 9.7 MB/s eta 0:00:00


# 3D CNNs

In [5]:
!pip install classification-models-3D
!pip install keras_applications
from classification_models_3D.tfkeras import Classifiers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.5/69.5 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 1.1 MB/s eta 0:00:00


In [6]:
import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow_addons as tfa
import glob, warnings
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
from collections import Counter
warnings.filterwarnings('ignore')
print('TensorFlow Version ' + tf.__version__)

/usr/local/lib/python3.10/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


TensorFlow Version 2.12.0


In [7]:
import zipfile
from io import BytesIO
from keras.layers import Dense, Activation, Flatten, Dropout, GlobalAveragePooling2D, Concatenate, Reshape, GlobalMaxPooling2D, GlobalMaxPooling3D, GlobalAveragePooling3D, Conv2D, Conv1D, Add
from keras import regularizers
from tensorflow.keras.optimizers import SGD, Adam
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from keras.callbacks import ReduceLROnPlateau
import sklearn
from sklearn.metrics import cohen_kappa_score, roc_auc_score, roc_curve,accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import random
import cv2
import gdown
import matplotlib.pyplot as plt
import requests
import tensorflow as tf
from sklearn.model_selection import train_test_split
import tensorflow_hub as hub
from PIL import Image
from sklearn.preprocessing import MinMaxScaler
from tensorflow import keras
import random
import sklearn
from sklearn.model_selection import StratifiedKFold, KFold
from tensorflow.keras import layers
import csv
from utils_GAMMA_V2 import padroniza_resultado
from utils_GAMMA_V2 import converte
from scipy import stats
from keras.applications.densenet import DenseNet121
from keras.applications.densenet import DenseNet169
from keras.applications.resnet import ResNet50
from keras.applications.resnet import ResNet101
from keras.applications.resnet import ResNet152
from keras.applications.inception_v3 import InceptionV3
from keras.applications.vgg19 import VGG19
from keras.models import Sequential, Model
import time
from pathlib import Path
import gc
from keras.backend import int_shape
import pickle
import optuna
from keras.applications.densenet import preprocess_input

# Hyperparameters

In [14]:
IMAGE_SIZE = 224
depth = 64
# TRAINING
EPOCHS = 15
early_stop_epochs = 2
learning_rate_epochs = 5
optimizer_direction = ['minimize', "maximize"]
#Foldes to save models and dataframes results
teste_number = '1/'
dir_save = '/content/Test_CNN/Models/Test_' + teste_number
results_dir = '/content/Test_CNN/Results_optuna/Test_' + teste_number
classification_results_dir = '/content/Test_CNN/Classification Results Optuna/Test_' + teste_number

# Read Fundus Images and Optic Disc Images (ROIs) - Train

In [9]:
from utils_GAMMA_V2 import load_fundus_images
folder_fundus = 'dataset_fundus/train/fundus_images flip - 100 images/'
folder_fundus_2 = 'ROI_disco_50px'

conj = 0# 0--> load train images; 1--> load test images
X_train_fundus,X_train_ROI,Y = load_fundus_images(path_dataset,folder_fundus,folder_fundus_2,conj,IMAGE_SIZE)
num_classes = len(np.unique(Y))

100


Read OCTs Volumes

In [10]:
from utils_GAMMA_V2 import process_scan
from utils_GAMMA_V2 import dir_octs
scan_paths = dir_octs(path_dataset,'multi-modality_images',0)#path to 3D volumes
X_oct = np.array([process_scan(path,IMAGE_SIZE, depth) for path in scan_paths])#read 3D volumes

Split Train/Validation

Fundus

In [10]:
x_train_fundus, x_val_fundus, y_train, y_val = train_test_split(X_train_fundus, Y, test_size=0.1, random_state=42,stratify=Y)

print(f"x_train shape: {x_train_fundus.shape} - y_train shape: {y_train.shape}")
print(f"x_val shape: {x_val_fundus.shape} - y_val shape: {y_val.shape}")

x_train shape: (90, 224, 224, 3) - y_train shape: (90,)
x_val shape: (10, 224, 224, 3) - y_val shape: (10,)


ROIs

In [16]:
x_train_ROI, x_val_ROI, y_train, y_val = train_test_split(X_train_ROI, Y, test_size=0.1, random_state=42,stratify=Y)

print(f"x_train shape: {x_train_ROI.shape} - y_train shape: {y_train.shape}")
print(f"x_val shape: {x_val_ROI.shape} - y_val shape: {y_val.shape}")

x_train shape: (90, 224, 224, 3) - y_train shape: (90,)
x_val shape: (10, 224, 224, 3) - y_val shape: (10,)


OCTs

In [ ]:
x_train_oct, x_val_oct, y_train, y_val = train_test_split(X_oct, Y, test_size=0.1, random_state=42,stratify=Y)

print(f"x_train shape: {x_train_oct.shape} - y_train shape: {y_train.shape}")
print(f"x_val shape: {x_val_oct.shape} - y_val shape: {y_val.shape}")

# Read Fundus Images and Optic Disc Images (ROIs) - Test

In [11]:
from utils_GAMMA_V2 import load_fundus_images
val_path_dataset = '/content/gdrive/My Drive/Doutorado/Bases/GAMMA_val_data/multi-modality_images/'
os.chdir(val_path_dataset)
folder_fundus = 'fundus_images'
folder_fundus_2 = 'ROI_disco_50px'
conj = 1# 1-->Load Images from test set
x_test_fundus,x_test_ROI = load_fundus_images(val_path_dataset,folder_fundus,folder_fundus_2,conj,IMAGE_SIZE)#
print(f"x_teste shape: {x_test_fundus.shape}")

100
x_teste shape: (100, 224, 224, 3)


Read OCT Volumes

In [ ]:
from utils_GAMMA_V2 import process_scan
from utils_GAMMA_V2 import dir_octs

val_path_dataset = '/content/gdrive/My Drive/Doutorado/Bases/GAMMA_val_data/multi-modality_images/'
os.chdir(val_path_dataset)
conj = 1#conj = 0 (Train); conj = 1(Test)
scan_paths = dir_octs(val_path_dataset,'multi-modality_images',conj)
#scan_paths = scan_paths[:1]
X_test_OCT = np.array([process_scan(path,IMAGE_SIZE, depth) for path in scan_paths])#read volumes

# Plot Fundus Images

In [ ]:
 # number of images
n_images = 100

# arrays
x = X_train_fundus
y = Y

# numer of rows and columns
rows = 10
cols = 10

# empty figure
fig, axs = plt.subplots(rows, cols, figsize=(28, 28))
axs = axs.ravel()

# para cada imagem
for i in range(n_images):
    #subplot
    axs[i].imshow(x[i], cmap='gray')
    axs[i].set_title(f'Label: {y[i]}')
    # remove as bordas e o eixo x, y
    axs[i].axis('off')

plt.tight_layout()
plt.show()

# Generate CSV to be evaluated online

In [12]:
def gera_csv(predicted_classes, classification_results_dir, trial_number):
  #classification_results_dir = 'content/Test_CNN/Classification Results Optuna/Test_'+teste_number
  if not Path(classification_results_dir).is_dir():
    os.mkdir(classification_results_dir)
  header = ['data','non','early','mid_advanced']
  results = padroniza_resultado(predicted_classes)
  #Alterar a pasta
  results_file_path = os.path.join(val_path_dataset, classification_results_dir  + '/Classification_Results_'+trial_number+'.csv')
  with open(results_file_path, 'w', encoding='UTF8', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(header)
    writer.writerows(results)

# Optimize One-level models; Inputs: Fundus or ROI Images

1 level

In [13]:
def objective(trial):
  num_dense_nodes = trial.suggest_categorical('num_dense_nodes', [64, 128, 256, 512])
  lr = trial.suggest_categorical('lr', [0.01, 0.001, 0.0001,])
  dense_nodes_divisor = trial.suggest_categorical('dense_nodes_divisor',[2, 4, 8])
  batch_size = trial.suggest_categorical('batch_size', [1, 2, 3])
  drop_out_rate=trial.suggest_float('drop_out_rate', 0.0, 0.5, step=0.1)
  cnn_model = trial.suggest_categorical('cnn_model', ['VGG19', 'resnet50', 'resnet101','resnet152','Densenet121','Densenet169'])
  num_layers = trial.suggest_categorical('num_layers',[1,2])#fully connected layers
  #i = tf.keras.layers.Input([IMAGE_SIZE,IMAGE_SIZE, 3], dtype = tf.uint8)
  #x = tf.cast(i, tf.float32)

  if cnn_model == 'InceptionV3':
    model_fundus = InceptionV3(weights='imagenet',
                        include_top=False,
                        input_shape=(IMAGE_SIZE,IMAGE_SIZE, 3))

  if cnn_model == 'VGG19':
    model_fundus = VGG19(weights='imagenet',
                        include_top=False,
                        input_shape=(IMAGE_SIZE,IMAGE_SIZE, 3))

  if cnn_model == 'resnet50':
    model_fundus = ResNet50(weights='imagenet',
                        include_top=False,
                        input_shape=(IMAGE_SIZE,IMAGE_SIZE, 3))

  if cnn_model == 'resnet101':
    model_fundus = ResNet101(weights='imagenet',
                        include_top=False,
                        input_shape=(IMAGE_SIZE,IMAGE_SIZE, 3))

  if cnn_model == 'resnet152':
    model_fundus = ResNet152(weights='imagenet',
                        include_top=False,
                        input_shape=(IMAGE_SIZE,IMAGE_SIZE, 3))

  if cnn_model == 'Densenet121':
    model_fundus = DenseNet121(weights='imagenet',
                        include_top=False,
                        input_shape=(IMAGE_SIZE,IMAGE_SIZE, 3))

  if cnn_model == 'Densenet169':
    model_fundus = DenseNet169(weights='imagenet',
                        include_top=False,
                        input_shape=(IMAGE_SIZE,IMAGE_SIZE, 3))


  x = model_fundus.output
  x = Flatten()(x)
  #MLP
  #x = Dense(num_dense_nodes, kernel_regularizer=regularizers.L1L2(l1=1e-5, l2=1e-4), activation='relu')(x)
  x = Dense(num_dense_nodes, activation='relu')(x)
  x = Dropout(drop_out_rate)(x)
  if num_layers == 2:
    #x = Dense(num_dense_nodes//dense_nodes_divisor, kernel_regularizer=regularizers.L1L2(l1=1e-5, l2=1e-4), activation='relu')(x)
    x = Dense(num_dense_nodes//dense_nodes_divisor, activation='relu')(x)
    x = Dropout(drop_out_rate)(x)

  output_tensor = Dense(num_classes, activation='softmax')(x)
  #Instantiate and compile model
  model = Model(inputs=model_fundus.input,outputs=output_tensor)

  #model = tf.keras.Model(inputs=[i], outputs=[output_tensor])
  opt = Adam(learning_rate=lr)  # default = 0.001 // learning_rate=lr
  model.compile(loss='sparse_categorical_crossentropy', optimizer=opt, metrics=['accuracy'])#loss='sparse_categorical_crossentropy'

  # callbacks for early stopping, learning rate reducer and model checkpoint
  fn = dir_save + str(trial.number) + '_cnn.h5'
  callbacks_list = [EarlyStopping(monitor='val_loss', patience=early_stop_epochs, mode='auto', min_delta=1e-3,verbose=1),
                     #MyThresholdCallback(threshold=0.95),
                    #ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=learning_rate_epochs, verbose=1, mode='auto', min_lr=1.0e-6),
                    ModelCheckpoint(filepath=fn, monitor='val_loss', verbose=1,save_best_only=True),
                    #PlotLossesKeras()
                    ]

  # fit the model (x_fundus or x_ROI)
  h = model.fit(x=x_train_ROI, y=y_train,
                #x=X_train_fundus, y = Y,#
                          batch_size=batch_size,
                          epochs=EPOCHS,
                          #validation_split=0.1,
                          validation_data = (x_val_ROI,y_val),
                          shuffle=True, verbose=1,
                          callbacks=callbacks_list)

  validation_loss = np.min(h.history['val_loss'])
  val_acc = np.max(h.history['val_accuracy'])

  #Evaluate on test set (define a treshold: val loss or acc )
  prob_class = model.predict(x_test_fundus)#probabilities
  predicted_classes = np.argmax(prob_class, axis=1)
  gera_csv(predicted_classes, classification_results_dir, str(trial.number))
  return validation_loss, val_acc

# Optimize One-level models; Inputs: 3D OCTs

In [ ]:
def objective(trial):
  num_dense_nodes = trial.suggest_categorical('num_dense_nodes', [32, 64, 128, 256])
  dense_nodes_divisor = trial.suggest_categorical('dense_nodes_divisor',[2, 4, 8])
  lr = trial.suggest_categorical('lr', [0.01, 0.001, 0.0001])
  batch_size = trial.suggest_categorical('batch_size', [1, 2, 3])
  drop_out_rate=trial.suggest_float('drop_out_rate', 0.0, 0.5, step=0.1)
  num_layers = trial.suggest_categorical('num_layers',[1,2])
  cnn_model = trial.suggest_categorical('cnn_model', ['VGG19', 'resnet50', 'resnet101','resnet152','Densenet121','Densenet169'])
  canais = 3

  if cnn_model == 'InceptionV3':
    Rede3D, preprocess_input = Classifiers.get('inceptionv3')
    model_oct = Rede3D(input_shape=(depth, IMAGE_SIZE, IMAGE_SIZE, canais), weights='imagenet',include_top=False)

  if cnn_model == 'VGG19':
    Rede3D, preprocess_input = Classifiers.get('vgg19')
    model_oct = Rede3D(input_shape=(depth, IMAGE_SIZE, IMAGE_SIZE, canais), weights='imagenet',include_top=False)

  if cnn_model == 'resnet50':
    Rede3D, preprocess_input = Classifiers.get('resnet50')
    model_oct = Rede3D(input_shape=(depth, IMAGE_SIZE, IMAGE_SIZE, canais), weights='imagenet',include_top=False)

  if cnn_model == 'resnet101':
    Rede3D, preprocess_input = Classifiers.get('resnet101')
    model_oct = Rede3D(input_shape=(depth, IMAGE_SIZE, IMAGE_SIZE, canais), weights='imagenet',include_top=False)

  if cnn_model == 'resnet152':
    Rede3D, preprocess_input = Classifiers.get('resnet152')
    model_oct = Rede3D(input_shape=(depth, IMAGE_SIZE, IMAGE_SIZE, canais), weights='imagenet',include_top=False)

  if cnn_model == 'Densenet121':
    Rede3D, preprocess_input = Classifiers.get('densenet121')
    model_oct = Rede3D(input_shape=(depth, IMAGE_SIZE, IMAGE_SIZE, canais), weights='imagenet',include_top=False)

  if cnn_model == 'Densenet169':
    Rede3D, preprocess_input = Classifiers.get('densenet169')
    model_oct = Rede3D(input_shape=(depth, IMAGE_SIZE, IMAGE_SIZE, canais), weights='imagenet',include_top=False)

  x = model_oct.output
  x = Flatten()(x)
  #MLP
  #x = Dense(num_dense_nodes, kernel_regularizer=regularizers.L1L2(l1=1e-5, l2=1e-4), activation='relu')(x)
  x = Dense(num_dense_nodes, activation='relu')(x)
  x = Dropout(drop_out_rate)(x)
  if num_layers == 2:
    #x = Dense(num_dense_nodes//dense_nodes_divisor, kernel_regularizer=regularizers.L1L2(l1=1e-5, l2=1e-4),activation='relu')(x)
    x = Dense(num_dense_nodes//dense_nodes_divisor, activation='relu')(x)
    x = Dropout(drop_out_rate)(x)
  output_tensor = Dense(num_classes, activation='softmax')(x)
  #Instantiate and compile model
  model = Model(inputs=model_oct.input,outputs=output_tensor)
  opt = Adam(learning_rate=lr)  # default = 0.001 // learning_rate=lr
  model.compile(loss='sparse_categorical_crossentropy', optimizer=opt, metrics=['accuracy'])#loss='sparse_categorical_crossentropy'

  # callbacks for early stopping and for learning rate reducer
  fn = dir_save + str(trial.number) + '_cnn.h5'
  callbacks_list = [EarlyStopping(monitor='val_loss', patience=early_stop_epochs, mode='auto',verbose=1),
                    #ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=learning_rate_epochs, verbose=1, mode='auto', min_lr=1.0e-6),
                    ModelCheckpoint(filepath=fn, monitor='val_loss',verbose=1, save_best_only=True),
                    ]

  # fit the model
  h = model.fit(x=x_train_oct, y=y_train,
                          batch_size=batch_size,
                          epochs=EPOCHS,
                          #validation_split=0.1,
                          validation_data = (x_val_oct,y_val),
                          shuffle=True, verbose=1,
                          callbacks=callbacks_list)

  validation_loss = np.min(h.history['val_loss'])
  val_acc = np.max(h.history['val_accuracy'])
  #Evaluate on test set (define a treshold: val loss or acc )
  prob_class = model.predict(x_test_fundus)#probabilities
  predicted_classes = np.argmax(prob_class, axis=1)
  gera_csv(predicted_classes, classification_results_dir, str(trial.number))
  return validation_loss, val_acc

#Optimization

In [17]:
val_path_dataset = '/content/Test_CNN/'
os.chdir(val_path_dataset)

if not Path(dir_save).is_dir():
  os.mkdir(dir_save)

print('\n*** starting at',pd.Timestamp.now())
start_time_total = time.time()
optuna.logging.set_verbosity(optuna.logging.WARNING)
study = optuna.create_study(directions=optimizer_direction,study_name="starter-experiment")
study.optimize(objective, n_trials=5,gc_after_trial=True)


*** starting at 2023-07-31 13:54:18.690283
234698864/234698864 [==============================] - 1s 0us/step
Epoch 1/15
30/30 [==============================] - ETA: 0s - loss: 93.2397 - accuracy: 0.3667
Epoch 1: val_loss did not improve from inf
30/30 [==============================] - 138s 265ms/step - loss: 93.2397 - accuracy: 0.3667 - val_loss: nan - val_accuracy: 0.5000
Epoch 2/15
30/30 [==============================] - ETA: 0s - loss: 16.2536 - accuracy: 0.4000
Epoch 2: val_loss improved from inf to 28263679282452690370560.00000, saving model to /content/Test_CNN/Models/Test_1/0_cnn.h5
30/30 [==============================] - 12s 398ms/step - loss: 16.2536 - accuracy: 0.4000 - val_loss: 28263679282452690370560.0000 - val_accuracy: 0.5000
Epoch 3/15
30/30 [==============================] - ETA: 0s - loss: 13.1013 - accuracy: 0.5000
Epoch 3: val_loss improved from 28263679282452690370560.00000 to 871057619681280.00000, saving model to /content/Test_CNN/Models/Test_1/0_cnn.h5
30/

[W 2023-07-31 13:58:56,803] Trial 0 failed with parameters: {'num_dense_nodes': 256, 'lr': 0.01, 'dense_nodes_divisor': 2, 'batch_size': 3, 'drop_out_rate': 0.4, 'cnn_model': 'resnet152', 'num_layers': 2} because of the following error: The value nan is not acceptable..
[W 2023-07-31 13:58:56,804] Trial 0 failed with value (nan, 0.5).


80134624/80134624 [==============================] - 1s 0us/step
Epoch 1/15
45/45 [==============================] - ETA: 0s - loss: 104502083584.0000 - accuracy: 0.3889
Epoch 1: val_loss improved from inf to 10348439.00000, saving model to /content/Test_CNN/Models/Test_1/1_cnn.h5
45/45 [==============================] - 13s 101ms/step - loss: 104502083584.0000 - accuracy: 0.3889 - val_loss: 10348439.0000 - val_accuracy: 0.5000
Epoch 2/15
45/45 [==============================] - ETA: 0s - loss: 169901296.0000 - accuracy: 0.3111
Epoch 2: val_loss improved from 10348439.00000 to 1065271.12500, saving model to /content/Test_CNN/Models/Test_1/1_cnn.h5
45/45 [==============================] - 5s 105ms/step - loss: 169901296.0000 - accuracy: 0.3111 - val_loss: 1065271.1250 - val_accuracy: 0.5000
Epoch 3/15
45/45 [==============================] - ETA: 0s - loss: 134409.6094 - accuracy: 0.3111
Epoch 3: val_loss improved from 1065271.12500 to 4318.79004, saving model to /content/Test_CNN/Model

In [18]:
# save results
df_results = study.trials_dataframe()
if not Path(results_dir).is_dir():
  os.mkdir(results_dir)
df_results.to_pickle(results_dir + 'df_optuna_results.pkl')
df_results.to_csv(results_dir + 'df_optuna_results.csv')
elapsed_time_total = (time.time()-start_time_total)/60
print('\n\ntotal elapsed time =',elapsed_time_total,' minutes')



total elapsed time = 19.304691219329833  minutes


# Sort results

In [19]:
import pandas as pd
val_path_dataset = '/content/Test_CNN/Results_optuna/Test_'+teste_number#path where optuna dataframe was saved
os.chdir(val_path_dataset)
dataframe = pd.read_csv('df_optuna_results.csv')
dataframe.sort_values("values_0",axis=0,ascending=True, inplace=True, na_position='first')
print("Sorted CSV file (according to multiple columns) = ", dataframe.head(10))

Sorted CSV file (according to multiple columns) =     Unnamed: 0  number    values_0  values_1              datetime_start  \
0           0       0         NaN       NaN  2023-07-31 13:54:18.691758   
2           2       2    1.069757       0.5  2023-07-31 13:59:54.031920   
3           3       3    1.074495       0.5  2023-07-31 14:05:58.178749   
4           4       4    2.402635       0.5  2023-07-31 14:11:54.931528   
1           1       1  965.798157       0.5  2023-07-31 13:58:58.287692   

            datetime_complete                duration  params_batch_size  \
0  2023-07-31 13:58:56.803408  0 days 00:04:38.111650                  3   
2  2023-07-31 14:05:57.191551  0 days 00:06:03.159631                  3   
3  2023-07-31 14:11:53.618867  0 days 00:05:55.440118                  2   
4  2023-07-31 14:13:36.128916  0 days 00:01:41.197388                  3   
1  2023-07-31 13:59:53.679613  0 days 00:00:55.391921                  2   

  params_cnn_model  params_dense_nodes_di

# Evaluate best models in split test

Test with fundus Images

In [27]:
best_model = keras.models.load_model('/content/Test_CNN/Models/Test_'+teste_number+'0_cnn.h5')
prob_class = best_model.predict(x_test_fundus)#probabilities
predicted_classes = np.argmax(prob_class, axis=1)
trial_number = 99
gera_csv(predicted_classes, classification_results_dir,str(trial_number))

4/4 [==============================] - 1s 101ms/step


# Ensemble of best models from optimization process

In [23]:
def make_final_predictions(xprod,
                           models_directory,
                           df_params,
                           threshold, num_models_accept,
                           optimization_direction):

    if optimization_direction == 'maximize':
        df_params.sort_values(by='values_1', ascending=False, inplace=True)
    else:
        df_params.sort_values(by='values_0', ascending=True, inplace=True)

    # apply threshold
    accepted_models_num = 0
    list_predicted_prob = []
    num_models_total = df_params.shape[0]
    for i in range(num_models_total):
        if optimization_direction == 'maximize':
            bool1 = df_params.loc[df_params.index[i],'values_1'] > threshold
        else:
            bool1 = df_params.loc[df_params.index[i],'values_0'] < threshold

        bool2 = df_params.loc[df_params.index[i],'state'] == 'COMPLETE'
        bool3 = accepted_models_num < num_models_accept
        if bool1 and bool2 and bool3:
            model_number = str(df_params.loc[df_params.index[i],'number'])
            #if model_number != '6':
            print (model_number)
            try:
                cnn_model = keras.models.load_model(models_directory + model_number + '_cnn.h5')
            except:
                print('\ncould not read model number:',model_number)
            else:
                list_predicted_prob.append(cnn_model.predict(xprod))
                accepted_models_num = accepted_models_num + 1

    # compute mean probabilities
    mean_probabilities = np.mean(list_predicted_prob, axis=0)

    # compute predicted class
    # argmax uses 1st ocurrance in case of a tie
    y_predicted_class = np.argmax(mean_probabilities, axis=1)
    return y_predicted_class

# fixed parameters - production (Ensemble)
threshold_error =  3# validation loss
number_of_models = 5

# get optuna results parameters
#models_dir = results_directory_stub + 'calibration/'
df_parameters = pd.read_pickle(results_dir + 'df_optuna_results.pkl')

#results_directory = results_directory_stub + calculation_type + '/'
#if not Path(results_directory).is_dir():
    #os.mkdir(results_directory)

final_result = make_final_predictions(x_test_fundus,
                               dir_save,
                               df_parameters,
                               threshold_error,
                               number_of_models, optimizer_direction)

trial_number = 100
gera_csv(final_result, classification_results_dir, str(trial_number))

2
4/4 [==============================] - 3s 178ms/step
3
4/4 [==============================] - 3s 173ms/step
4
4/4 [==============================] - 2s 115ms/step


# Combine best Models

In [ ]:
#Used in ensemble
x_test_list=[]
#Images
x_test_list.append(x_test_fundus)
x_test_list.append(x_test_ROI)
x_test_list.append(X_test_OCT)

In [ ]:
def ensemble_results(cnn_model,x):
    return cnn_model.predict(x)

In [ ]:
#best load model with background image as input
#best_model_fundus = keras.models.load_model('Modelos_CNN_Optuna/Testes_50_Trials/Teste_1/22_cnn.h5')#VGG19
models_list = []
best_model_fundus = keras.models.load_model('Modelos_CNN_Optuna/Teste_11/32_cnn.h5')#Dense169 - 224x224
models_list.append(best_model_fundus)
#load best model with ROI as input
best_model_ROI = keras.models.load_model('Modelos_CNN_Optuna/Testes_50_Trials/Teste_2A/16_cnn.h5')#VGG19 - 128x128
models_list.append(best_model_ROI)
#load better model with OCT as input
best_model_OCT = keras.models.load_model('Modelos_CNN_Optuna/Testes_30_Trials/Teste_3/11_cnn.h5')#Dense121 - 3D
models_list.append(best_model_OCT)

In [ ]:
list_predicted_prob = []
for i in range (len(models_list)):
  list_predicted_prob.append(ensemble_results(models_list[i], x_test_list[i]))

4/4 [==============================] - 126s 27s/step


Average Ensemble

In [ ]:
# compute mean probabilities
mean_probabilities = np.mean(list_predicted_prob, axis=0)

# compute predicted class
# argmax uses 1st ocurrance in case of a tie
y_predicted_class = np.argmax(mean_probabilities, axis=1)
#gera_csv
gera_csv(y_predicted_class, classification_results_dir,str(trial_number))

Mode ensemble

In [ ]:
## function to identify cases where 2 models predicted in a class
def conta_votos_2x1(v1,v2,v3):
  votos = []
  # 1, 2 ou 3
  index1 = v1.index('1')#model fundus stage predicition
  votos.append(index1)
  index2 = v2.index('1')#model roi stage predicition
  votos.append(index2)
  index3 = v3.index('1')#model oct stage predicition
  votos.append(index3)

  #verifica quem teve 2 votos
  if votos.count(1) == 2:
    a=[v1[0],'1','0','0']
    return (a)
  if votos.count(2) == 2:
    a=[v1[0],'0','1','0']
    return (a)
  if votos.count(3) == 2:
    a=[v1[0],'0','0','1']
    return (a)

In [ ]:
#function gives the following priority to resolve conflicts
# if the oct models gives an initial diagnosis result, it will be kept
# if the fundus model is advanced or normal it will be kept
# otherwise the fundus model result will be kept
def analisa_conflito(v1,v2,v3):
  votos = []
  index1 = v1.index('1')
  votos.append(index1)
  index2 = v2.index('1')
  votos.append(index2)
  index3 = v3.index('1')
  votos.append(index3)

  if index3 == 2: #result of oct model is early glaucoma
    return ([v1[0],'0','1','0'])
  else:
    return v1

In [ ]:
import csv
import pprint

#extract the data from the csv file and put it in a list
fundus_predictions = []
roi_predictions = []
oct_predictions = []
#ensemble = []
#csv of best models
results_fundo = os.path.join(path_dataset, 'Melhores resultados/Classification_Results_3_Trial_32.csv')#classification of fundus model
results_roi = os.path.join(path_dataset, 'Melhores resultados/Classification_Results_3_Trial_16.csv')#classification of roi model
results_oct = os.path.join(path_dataset, 'Melhores resultados/Classification_Results_2_Trial_11.csv')#classification of oct model

with open(results_fundo,'r') as f:
    lines=csv.reader(f)
    for key, line in enumerate(lines):
        fundus_predictions.append(line)

with open(results_roi,'r') as f:
    lines=csv.reader(f)
    for key, line in enumerate(lines):
        roi_predictions.append(line)

with open(results_oct,'r') as f:
    lines=csv.reader(f)
    for key, line in enumerate(lines):
        oct_predictions.append(line)

In [ ]:
ensemble = []
for i in range(len(fundus_predictions)):
  if i == 0:
    ensemble.append(fundus_predictions[i])#Name of each column
  elif fundus_predictions[i] == roi_predictions[i] and roi_predictions[i] == oct_predictions[i]:#All models predict same class
    ensemble.append(fundus_predictions[i])
  elif fundus_predictions[i] != roi_predictions[i] and roi_predictions[i] != oct_predictions[i] and fundus_predictions[i] != oct_predictions[i]:#each model predicts a different class
    ensemble.append(fundus_predictions[i])#keeps fundus model result
    #ensemble.append(analisa_conflito(fundus_predictions[i], roi_predictions[i], oct_predictions[i]))#uses function to determine final result
  else:
    ensemble.append(conta_votos_2x1(fundus_predictions[i], roi_predictions[i], oct_predictions[i]))#checks cases where 2 models predicted the same class

#create a csv
results_file_path = os.path.join(val_path_dataset, "./Classification Results Optuna/Ensemble/Classification_Results.csv")
with open(results_file_path, 'w', encoding='UTF8', newline='') as f:
  writer = csv.writer(f)
  #writer.writerow(header)
  writer.writerows(ensemble)